In [0]:
%sql
-- Get current max ProcessQueueId
WITH max_queue_id AS (
    SELECT COALESCE(MAX(ProcessQueueId), 0) AS max_id
    FROM control.processqueue
),
-- Get active tables and assign ProcessInstanceId
active_tables AS (
    SELECT
        td.TableName,
        row_number() OVER (ORDER BY td.TableName)
        + (SELECT COALESCE(MAX(ProcessInstanceId), 0) FROM control.processqueue)
        AS ProcessInstanceId
    FROM metadata.tabledetails td
    WHERE td.IsActive = 1
),
-- Get active stages
active_stages AS (
    SELECT StageId
    FROM metadata.stagemapping
    WHERE IsActive = 1
),
-- Create Table × Stage combinations
table_stage_matrix AS (
    SELECT
        t.TableName,
        t.ProcessInstanceId,
        s.StageId
    FROM active_tables t
    CROSS JOIN active_stages s
),
-- Generate unique ProcessQueueId per row
final_rows AS (
    SELECT
        row_number() OVER (ORDER BY TableName, StageId)
        + (SELECT max_id FROM max_queue_id) AS ProcessQueueId,
        ProcessInstanceId,
        TableName,
        CAST(NULL AS TIMESTAMP) AS QueuedTime,
        StageId,
        'AwaitingProcessing' AS ProcessStatus,
        CAST(NULL AS TIMESTAMP) AS ProcessStartTime,
        CAST(NULL AS TIMESTAMP) AS ProcessEndTime,
        CAST(NULL AS BIGINT) AS ProcessDuration
    FROM table_stage_matrix
)
-- Insert rows into the processqueue table
INSERT INTO control.processqueue
SELECT * FROM final_rows;